In [6]:
import pandas as pd

In [2]:
pd.read_csv('sniffed_df.csv')

,Unnamed: 0,interface,Time,Source,Destination,Length
0,0,en0,1.677752e+09,00:23:eb:f9:60:c0,78:31:c1:bf:c0:76,118
1,1,en0,1.677752e+09,78:31:c1:bf:c0:76,00:00:0c:07:ac:4c,235
2,2,en0,1.677752e+09,78:31:c1:bf:c0:76,00:00:0c:07:ac:4c,1304
3,3,en0,1.677752e+09,78:31:c1:bf:c0:76,00:00:0c:07:ac:4c,1137


In [1]:
import netifaces
from scapy.all import *
import pandas as pd
import time

In [2]:
# On garde que les ports wifi et ethernet
interfaces = list(filter(lambda s: ('en' or 'eth') in s, netifaces.interfaces()))

# Capturer les packets pendant 20 secondes
pkt = sniff(iface=interfaces, count=100)

data = []
for packet in pkt:
    if IP in packet : # garder seulement packet IP
        data.append([packet.sniffed_on ,packet.time, packet[IP].src, packet[IP].dst, len(packet)])

sniffed_df = pd.DataFrame(data, columns=['interface','Time', 'Source', 'Destination', 'Length'])

#Filtrer sur l'interface la plus utilisée
most_used_interface = sniffed_df.groupby(by = 'interface').sum('Length').nlargest(1, 'Length').iloc[0].name
sniffed_df = sniffed_df.where(sniffed_df["interface"] == most_used_interface).drop("interface")




In [65]:
data = []
cpt = 0
for packet in pkt:
    if IP in packet : # garder seulement packet IP
        data.append([packet.sniffed_on ,packet.time, packet[IP].src, packet[IP].dst, len(packet)])

sniffed_df = pd.DataFrame(data, columns=['interface','Time', 'Source', 'Destination', 'Length'])

In [71]:
ip = get_if_addr(most_used_interface)
sniffed_df['outbound'] = sniffed_df['Source'].apply(lambda x : x == ip)

In [73]:
sniffed_df['delta'] = sniffed_df.Time.diff()
window = 4 
sniffed_df['ra_delta'] = sniffed_df.delta.rolling(window=window).mean()

,interface,Time,Source,Destination,Length,outbound,delta
0,en0,1.677934e+09,192.168.1.27,17.248.177.39,105,True,NaN
1,en0,1.677934e+09,192.168.1.27,17.248.177.39,90,True,0.004093
2,en0,1.677934e+09,192.168.1.27,17.248.177.39,66,True,0.005720
3,en0,1.677934e+09,192.168.1.27,142.250.13.128,78,True,0.002986
4,en0,1.677934e+09,17.248.177.39,192.168.1.27,66,False,0.016744
...,...,...,...,...,...,...,...
91,en0,1.677934e+09,192.168.1.27,17.248.176.233,66,True,0.000081
92,en0,1.677934e+09,17.248.176.233,192.168.1.27,66,False,0.001926
93,en0,1.677934e+09,192.168.1.27,13.89.178.26,78,True,0.327506
94,en0,1.677934e+09,13.89.178.26,192.168.1.27,66,False,0.131037
